한국어 영화 리뷰 - BERT

In [30]:
# https://drive.google.com/file/d/1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o/view?usp=sharing

import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)

In [31]:
# rating 6 이상이면 긍정 라벨 생성 y로 저장
import numpy as np
from sklearn.model_selection import train_test_split

# y를 생성해서 review 컬럼이 x 데이터 분할 데이터 개수 확인
y = np.array([1 if value >= 6 else 0 for value in df.rating])  # 6 이상이면 1(긍정), 미만이면 0(부정)
x = df['review']  # 리뷰 텍스트

# 데이터셋을 학습 검증 평가로 나눈다 x_train x_val x_test

X_, x_test, y_, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
x_train, x_val, y_train , y_val = train_test_split(X_, y_, test_size=0.2, random_state=42, stratify=y_)

print(x_train.shape,x_val.shape, x_test.shape)
print(y_train.shape,y_val.shape, y_test.shape)

(9424,) (2356,) (2945,)
(9424,) (2356,) (2945,)


In [32]:
import torch
# from datasets import load_metric # 2022 이후로 huggingface에서 deprecated evaluate
# load_metric이 datasets에서 제거됨 - evaluate 라이브러리 사용
import evaluate
metric = evaluate.load('accuracy')

# 필요 시 설치: pip install evaluate


In [33]:
def compute_metrics(eval_pred):
    '''
    Args:
        eval_pred : logits.labels를 가지고 있는 dataset
    Returns:
        accuracy
    '''
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [34]:
# 이 코드는 PyTorch의 사용자 정의 데이터셋(Dataset) 클래스를 정의합니다.
# PyTorch에서 딥러닝 모델을 훈련할 때 데이터를 효율적으로 로드하고 처리하기 위해 사용됩니다.

# `torch.utils.data.Dataset`을 상속받아 `OurDataset` 클래스를 만듭니다.
# 이 클래스는 다음 세 가지 필수 메서드를 구현해야 합니다:

# 1. `__init__(self, encodings, labels)`:
#    - 데이터셋을 초기화하는 생성자입니다.
#    - `encodings`: 입력 데이터(예: 텍스트 데이터의 토큰화된 인코딩)를 담고 있는 딕셔너리입니다.
#      각 키는 'input_ids', 'attention_mask' 등과 같을 수 있으며, 값은 텐서 또는 리스트 형태입니다.
#    - `labels`: 각 입력 데이터에 해당하는 레이블(정답)을 담고 있는 리스트 또는 텐서입니다.
#    - 이 메서드에서는 전달받은 `encodings`와 `labels`를 클래스 인스턴스의 속성으로 저장합니다.

# 2. `__getitem__(self, idx)`:
#    - 특정 인덱스 `idx`에 해당하는 데이터 샘플을 반환하는 메서드입니다.
#    - `item = {key: torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}`:
#      `self.encodings` 딕셔너리에서 `idx`에 해당하는 값을 가져와 PyTorch 텐서로 변환합니다.
#      `.clone().detach()`는 원본 데이터에 대한 참조를 끊고 독립적인 복사본을 만들어,
#      데이터가 모델 훈련 중 의도치 않게 변경되는 것을 방지하고 메모리 효율성을 높일 수 있습니다.
#    - `item['labels'] = torch.tensor(self.labels[idx])`:
#      `self.labels`에서 `idx`에 해당하는 레이블을 가져와 PyTorch 텐서로 변환하여 `item` 딕셔너리에 추가합니다.
#    - 최종적으로, 입력 데이터와 해당 레이블을 포함하는 딕셔너리 `item`을 반환합니다.

# 3. `__len__(self)`:
#    - 데이터셋의 전체 크기(샘플 수)를 반환하는 메서드입니다.
#    - `self.labels`의 길이를 반환하여 데이터셋에 몇 개의 샘플이 있는지 알려줍니다.

# 요약: 이 `OurDataset` 클래스는 토큰화된 텍스트 인코딩과 해당 레이블을 받아,
# PyTorch `DataLoader`가 개별 샘플을 쉽게 가져올 수 있도록 표준화된 인터페이스를 제공합니다.
# 이는 특히 Hugging Face Transformers 라이브러리와 같은 곳에서 모델 훈련을 위한 데이터 준비에 유용합니다.

# 데이타셋 생성 클래스(상속)
# __init__ __getitem__ __len__  
# x,y 각각 텐서  y의 개수

import torch
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels    = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # CUDA 호환성을 위해 long 타입 명시
        return item
    def __len__(self):
        return len(self.labels)

In [35]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased') #다국어 지원(한국어)
print(tokenizer.tokenize("안녕하세요. 반갑습니다."))

['안', '##녕', '##하', '##세', '##요', '.', '반', '##갑', '##습', '##니다', '.']


In [36]:
# `return_tensors="pt"`는 토크나이저가 처리된 입력을 PyTorch 텐서 형식으로 반환하도록 지시합니다.
# 다른 옵션으로는 `tf` (TensorFlow 텐서), `np` (NumPy 배열), 또는 `None` (Python 리스트) 등이 있습니다.
inputs = tokenizer("안녕하세요. 반갑습니다.", return_tensors="pt")
print(inputs)

{'input_ids': tensor([[   101,   9521, 118741,  35506,  24982,  48549,    119,   9321, 118610,
         119081,  48345,    119,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [37]:
# 2022년 이후로는 Auto~~~~ 토크나이져와 모델을 사용하도록 권장(벤더사에서 업데이트시 유리)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
print(tokenizer.tokenize("안녕하세요. 반갑습니다."))
inputs = tokenizer("안녕하세요. 반갑습니다.")
print(inputs)

['안', '##녕', '##하', '##세', '##요', '.', '반', '##갑', '##습', '##니다', '.']
{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [39]:
# mBERT + Trainer로 미세조정(Fine-Tuning)
# AutoModelForSequenceClassification: 시퀀스 분류(예: 감성 분석, 스팸 감지)를 위한 사전 학습된 모델을 로드하는 데 사용됩니다.
# TrainingArguments: Trainer 클래스에 전달될 훈련 관련 하이퍼파라미터 및 설정을 정의하는 데 사용됩니다.
# Trainer: Hugging Face Transformers 라이브러리에서 모델 훈련 및 평가를 위한 고수준 API를 제공합니다.
# AutoTokenizer: 사전 학습된 모델에 해당하는 토크나이저를 자동으로 로드하는 데 사용됩니다. 텍스트를 모델 입력 형식으로 변환합니다.
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer

# 토큰화
train_input = tokenizer(x_train.tolist(), padding=True, truncation=True, return_tensors='pt')
val_input = tokenizer(x_val.tolist(), padding=True, truncation=True, return_tensors='pt')
test_input = tokenizer(x_test.tolist(), padding=True, truncation=True, return_tensors='pt')
print(train_input.keys())

# Dataset생성 (레이블을 리스트로 변환)
train_dataset = OurDataset(train_input, y_train.tolist())
val_dataset = OurDataset(val_input, y_val.tolist())
test_dataset = OurDataset(test_input, y_test.tolist())

# 분류모델 생성 - 토크나이저와 동일한 multilingual 모델 사용
model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased', 
    num_labels=2  # 이진 분류이므로 2개 클래스
)

# Trainer에 사용할 Argument 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,  # 배치 크기 축소로 메모리 안정성 확보
    per_device_eval_batch_size=16,
    warmup_steps=100,  # warmup_steps 축소
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_steps=50,
    load_best_model_at_end=True,
    report_to='none',
    dataloader_drop_last=True  # 불완전한 배치 제거
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()



dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [40]:
# 새 셀에 추가
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
#  데이타셋 생성 클래스(상속)

In [ ]:
# %pip install evaluate

In [ ]:
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments, 
    Trainer
)
import torch.nn.functional as F
from torch.utils.data import Dataset